In [4]:
!pip install firebase-admin
!pip install firebase


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
^C
ERROR: Operation cancelled by user


In [5]:
import firebase_admin
from firebase_admin import credentials, firestore
from google.oauth2 import service_account
import json
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [6]:
with  open('creds.json') as f:
    service_account_info = json.load(f)

credentials = service_account.Credentials.from_service_account_info(service_account_info)
db = firestore.Client(credentials=credentials)

In [14]:
# reading the CSV of courses
df = pd.read_csv('course_data.csv')
df = df.drop(columns=['sem_last_laught'])

df

,id,title,description,number,average_rating,num_reviews,mnemonic
0,1,Introduction to African-American and African S...,This introductory course surveys the histories...,1010,4.633333,10,AAS
1,5,Intermediate Seminar in African-American & Afr...,"Reading, class discussion, and written assignm...",3500,3.925926,9,AAS
2,29,Introduction to American Studies,This course introduces students to American St...,2001,3.552083,32,AMST
3,31,Theories and Methods of American Studies,This seminar course will introduce majors to v...,3001,4.196078,17,AMST
4,32,Introduction to Asian American Studies,An interdisciplinary introduction to the cultu...,3180,4.142857,7,AMST
...,...,...,...,...,...,...,...
990,15550,Psychopharmacology of Plants,This course begins with discussion of pharmaco...,4019,3.000000,1,BIOL
991,15584,Latin Linguistics,This course will examine some of the major iss...,5080,4.000000,1,LATI
992,15602,Questions in the Study of Religion,This course introduces undergraduates to the e...,1000,4.666667,1,RELG
993,15606,Languages of Love in South Asia: Bhakti and Be...,The course explores some of the most influenti...,3350,4.333333,1,SATR


In [15]:
courses = db.collection('courses')
def upload_course(row):
  dict_row = row.to_dict()
  
  try:
    courses.document(str(dict_row['id'])).set(dict_row)
  except:
    print("failed to set", dict_row['id'])

df.progress_apply(upload_course, axis=1)

100%|██████████| 995/995 [01:15<00:00, 13.26it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
990    None
991    None
992    None
993    None
994    None
Length: 995, dtype: object

In [ ]:
import requests

session = requests.Session()

for club in tqdm(db.collection('clubs').stream()):
  # add a description field
  club_dict = club.to_dict()

  if club.get('description'):
    continue
  
  club_dict['description'] = ''

  uri = club_dict.get('uri')
  
  if not uri:
    continue

  url = f"https://api.presence.io/virginia/v1/organizations/{uri}"
  response = session.get(url)

  if response.status_code == 200:
    data = response.json()
    club_dict['description'] = data.get('description')
    club.reference.set(club_dict)

  else:
    print("failed to get", uri)

0it [00:00, ?it/s]

945it [00:08, 109.03it/s]


KeyError: "'description' is not contained in the data"

In [17]:
!pip install -qU \
  pinecone-client==3.0.0 \
  pandas==2.0.3

from pinecone import Pinecone, ServerlessSpec
import time
import os

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY')

# configure client
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec(cloud='aws', region='us-west-2')

# pc.create_index(
#     name="clubs", 
#     dimension=1536, 
#     metric="euclidean",
#     spec=spec
# )

pc.create_index(
    name="courses", 
    dimension=1536, 
    metric="euclidean",
    spec=spec
)


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
from openai import OpenAI

OPENAIKEY = os.environ.get('OPENAI_API_KEY') 
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAIKEY,
)

In [ ]:
from bs4 import BeautifulSoup

def get_embedding(club, model="text-embedding-ada-002"): 
  # parse the description
  description = club.get('description')
  if not description:
    return

  soup = BeautifulSoup(description, 'html.parser')

  prompt = f"""
  name {club.get('name')}
  description {soup.text}
  tags {", ".join(list(club.get('categories')) if club.get('categories') else [])}
  members {club.get('memberCount')}
  """

  prompt = prompt.strip().replace("\n", " ").strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ")
  print(prompt)
  # def get_embedding(text, model="text-embedding-ada-002"):
  #  text = text.replace("\n", " ")[:32_000]
  return client.embeddings.create(input = [prompt], model=model).data[0].embedding

upsertion = [
  # {'id': id, 'values': embedding}
]

for club in tqdm(db.collection('clubs').stream()):
  club_dict = club.to_dict()

  embedding = get_embedding(club_dict)
  if embedding:
    upsertion.append({'id': club.id, 'values': embedding})

0it [00:00, ?it/s]

name Net Impact at the University of Virginia description Net Impact focuses on the intersection of sustainability, social impact, and business. We aim to create a positive impact on Grounds by raising awareness of careers in sustainability, educating students by offering events such as dinner series with Professors and panels with business leaders, providing a community for like-minded individuals, and equipping students with the necessary tools to intertwine business with sustainability and social impact. tags McIntire School of Commerce, Sustainability, Academic & Professional, Contracted Independent Organization (CIO) members 22


2it [00:00,  4.51it/s]

name Virginia Players description Players tags Department Group or Program, Social & Hobby, Visual & Performing Arts members 12
name Accessible Theatre Project description Accessible Theatre Project strives to offer safe, judgement-free environments for audiences who would benefit from accessibility accommodations. We provide inclusive theater experiences for neurodiverse audience members and their families by partnering with theater organizations to create sensory-friendly productions. tags Contracted Independent Organization (CIO), Visual & Performing Arts, Social & Hobby, Public Service, No Application or Interview Required members 11


4it [00:00,  5.82it/s]

...

name Alpha Phi (Zeta Iota Deuteron Chapter) description FSL tags Social & Hobby, Political & Advocacy, Public Service, Fraternity or Sorority members 2
name Daniel Hale Williams Pre-Health Honor Society description Daniel Hale Williams was the first African-American Cardiologist, and the first person in the United States to perform a successful open heart surgery. He was also the founder of Provident Hospital in Chicago, which was the first non-segregated hospital in the United States.The mission of the Daniel Hale Williams Pre-Health Honor Society (DHW) is to increase the number of culturally capable and sensitive Black and other ethnic minority physicians, by furthering their academic and clinical success as premedical students through: recruitment, academic counseling and support, curriculum guidance, peer support, and post-graduate assistance.DHW is also affiliated with the Student National Medical Association (SNMA), which is affiliated with the National Medical Association (NMA).

943it [02:45,  6.88it/s]

name Darden Jewish Student Association description The Darden Jewish Student Association (DJSA) seeks to further the awareness of Jewish culture, tradition, and history within the Darden Community through educational and social avenues. The club aims to create a safe space for Jews at Darden, as well as in the Charlottesville community, through engaging in discussions, celebrations, and gatherings of all avenues. DJSA also strives to engage non-Jewish allies to educate them, integrate them into the Jewish community, and fight antisemitism, whether it’s within the Darden community, the greater Charlottesville area, or beyond. The DJSA cultivates and supports a friendly community for all students to congregate, converse, and learn at Darden, while strengthening relationships with other Jewish groups and organizations in Charlottesville and nationally. tags No Application or Interview Required, Religious & Spiritual, Darden School, Cultural & Ethnic, Contracted Independent Organization (C

989it [02:45,  5.97it/s]

name The AcHOOstics description We are the University of Virginia’s only philanthropic a cappella group, founded in 2015 by a group of women who wanted to combine their love of music with a love of service. Every semester, a portion of our proceeds from concert ticket sales and fundraisers go to a charity selected each semester, such as the Sexual Assault Resource Agency or the Music Resource Center. We perform at multiple events across Grounds, including Rotunda Sing, Lighting of the Lawn, baseball and basketball games singing the National Anthem, and fundraisers such as Pancakes for Parkinson’s, as well as in the community with performances at local nursing homes and the UVA Children’s Hospital. We have also been invited to perform at multiple a cappella events hosted by groups at other universities and enjoy establishing relationships with groups like ours across the country. Our group brings together passionate women with diverse personalities, experiences, and voices to give back 

In [ ]:
# upload data
index = pc.Index("clubs")

index.upsert(vectors=upsertion[700:])
# upsertion

{'upserted_count': 206}

In [20]:
courses = pc.Index("courses")

course_upsertion = []

def get_course_embedding(course_dict: dict): 
  title = course_dict.get('title')
  description = course_dict.get('description')
  mneumonic = course_dict.get('mneumonic') 


  prompt = f"""
  title {title}
  description {description}
  mneumonic {mneumonic}
  """

  prompt = prompt.strip().replace("\n", " ").strip().replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ")

  return client.embeddings.create(input = [prompt], model="text-embedding-ada-002").data[0].embedding

for course in tqdm(db.collection('courses').stream(), total=df.shape[1]):
  course_dict = course.to_dict()

  embedding = get_course_embedding(course_dict)
  if embedding:
    course_upsertion.append({'id': course.id, 'values': embedding})

995it [02:35,  6.39it/s]                     


In [27]:
# course_upsertion
courses.upsert(vectors=course_upsertion[:200])


{'upserted_count': 200}